In [58]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Activation, Concatenate
from keras import Sequential

from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import SGD,Adam
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

#Different Transfer learning rate/models
#https://keras.io/api/applications/
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [59]:
train_dir = "grayscale"
#test_dir = "../input/test/test/"
train_df = pd.read_csv('file_types.csv')
train_df['ImageName'] = train_df['ImageName'].astype(str)
train_df['ImageName'] = train_df['ImageName'].astype(str).str.zfill(6)
train_df['Class'] = pd.factorize(train_df['FileType'])[0] 
train_df.head()

,ImageName,FileType,Class
0,000008,ppt,0
1,000027,csv,1
2,000039,xml,2
3,000067,xml,2
4,000081,txt,3


In [60]:
width = 100
height = 100

In [61]:
X_tr = []
Y_tr = []
imges = train_df['ImageName'].values
for img_id in imges:
    file = os.path.join(train_dir, img_id) + '.png'
   
    temp = cv2.imread(file) 
            
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = np.expand_dims(temp, axis=2)
    X_tr.append(temp)    
    Y_tr.append(train_df[train_df['ImageName'] == img_id]['Class'].values[0])  
X_tr = np.asarray(X_tr)
Y_tr = to_categorical(Y_tr)
#X_tr = X_tr.astype('float32')
#X_tr /= 255
#Y_tr = np.asarray(Y_tr)
print((X_tr.shape,Y_tr.shape))

((15797, 100, 100, 1), (15797, 14))


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
    test_size=0.20, shuffle = True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,
    test_size=0.50, shuffle = True)

In [63]:
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)
print('X_val: ', X_val.shape)
print('y_val: ', y_val.shape)

X_train:  (4528, 100, 100, 1)
y_train:  (4528, 14)
X_test:  (566, 100, 100, 1)
y_test:  (566, 14)
X_val:  (566, 100, 100, 1)
y_val:  (566, 14)


In [64]:
batch_size= 32
epochs=100
learn_rate=.0001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

C:\Users\Razaq\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [65]:
train_dategen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = train_dategen.flow(X_train,  y_train, batch_size= batch_size)

validation_dategen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validation_generator = validation_dategen.flow(X_val, y_val, batch_size= batch_size)

test_generator = ImageDataGenerator(rescale = 1./255.)

In [66]:
#Learning Rate Annealer
lrr= ReduceLROnPlateau(monitor='val_accuracy', factor=.01, patience=3, min_lr=1e-5) #change learning rATE  to get the best result

In [67]:
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
#mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [68]:
#Defining the googlenet model
#use resnet - try various versions of resnet
#compare googlenet to resnet and the other 4, to see which one is better. use the latest version of them

img_width = 100
img_height = 100

base_model = InceptionV3(include_top = False, weights = None, input_shape = (img_width,img_height,1))
#base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3)) #
#base_model = ResNet101(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
#base_model = Xception(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height, 3))
#base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
#base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,1))


#use resnet - try various versions of resnet
#compare googlenet to resnet and the other 4, to see which one is better. use the latest version of them

#changing the last layer
for layer in base_model.layers:
    layer.trainable = False
#https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a

# #changing the last layer
# for layer in base_model.layers:
#   layer.trainable = False

In [69]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(14, activation='softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802208  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 14)                28686     
                                                                 
Total params: 21,830,894
Trainable params: 28,686
Non-trainable params: 21,802,208
_________________________________________________________________


In [70]:
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [71]:
history = model.fit_generator(train_generator, 
                    epochs = epochs, 
                    steps_per_epoch = X_train.shape[0]//batch_size, 
                    validation_data = validation_generator, 
                    validation_steps = X_val.shape[0]//batch_size, callbacks=[lrr],  verbose = 1)

Epoch 1/100


C:\Users\Razaq\AppData\Local\Temp\ipykernel_10020\3948111818.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


141/141 [==============================] - 26s 161ms/step - loss: 2.6329 - accuracy: 0.2340 - val_loss: 2.6265 - val_accuracy: 0.2316 - lr: 1.0000e-04
Epoch 2/100
141/141 [==============================] - 24s 167ms/step - loss: 2.6190 - accuracy: 0.2471 - val_loss: 2.6134 - val_accuracy: 0.2279 - lr: 1.0000e-04
Epoch 3/100
141/141 [==============================] - 22s 155ms/step - loss: 2.6054 - accuracy: 0.2456 - val_loss: 2.6010 - val_accuracy: 0.2298 - lr: 1.0000e-04
Epoch 4/100
141/141 [==============================] - 22s 154ms/step - loss: 2.5918 - accuracy: 0.2451 - val_loss: 2.5876 - val_accuracy: 0.2316 - lr: 1.0000e-04
Epoch 5/100
141/141 [==============================] - 21s 152ms/step - loss: 2.5838 - accuracy: 0.2456 - val_loss: 2.5855 - val_accuracy: 0.2279 - lr: 1.0000e-05
Epoch 6/100
141/141 [==============================] - 22s 153ms/step - loss: 2.5826 - accuracy: 0.2449 - val_loss: 2.5846 - val_accuracy: 0.2298 - lr: 1.0000e-05
Epoch 7/100
141/141 [=============

In [72]:
y_pred = model.predict(X_test)

In [73]:
y_test

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [74]:
cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
cm

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  30,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   8,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  10,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 124,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   5,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  11,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   4,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  33,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 137,
          0],
       [  0,   0,   0,   0,   0,   0,   0, 

In [75]:

from sklearn.metrics import classification_report
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        30
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00       124
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00        33
          10       0.00      0.00      0.00       137
          11       0.00      0.00      0.00        23
          12       0.25      1.00      0.39       139
          13       0.00      0.00      0.00        38

    accuracy                           0.25       566
   macro avg       0.02      0.07      0.03       566
weighted avg       0.06      0.25      0.10       566



C:\Users\Razaq\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Razaq\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Razaq\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,